In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pystan
##import arviz as az

##import nest_asyncio
##nest_asyncio.apply()

In [5]:
# Open the Stan file
with open('diffusion_model.stan', 'r') as file:
    stan_code = file.read()

# Print the Stan code
print(stan_code)

data {
    int<lower=1> N;
    array[N] real<lower=0> y;
    array[N] int<lower=1, upper=2> condition;
    array[N] int<lower=0, upper=1> choice;
}

parameters {
    // Your code here
}

model {
    // Priors
    // Your code here

    // Likelihood
    for (n in 1:N) {
        // Condition 1
        if (condition[n] == 1) {
            if (choice[n] == 1) {
                 // Your code here
            }
            else {
                 // Your code here
            }
        }
        // Condition 2
        if (condition[n] == 2) {
            if (choice[n] == 1) {
                // Your code here
            }
            else {
                 // Your code here
            }
        }
    }
}


In [20]:
# Import the CSV file into a pandas DataFrame
df = pd.read_csv('sample_response_times.csv', delimiter=';',dtype={'rt': float, 'choice': int, 'condition': int})

In [24]:
##Program code for stan model. Contains all of the parameter specifications
program_code="""
    data {
        int<lower=1> N;
        real y[N];
        int<lower=1, upper=2> condition[N];  
        int<lower=0, upper=1> choice[N]; 
    }

    parameters {
        real<lower=0> v1;
        real<lower=0> v2;
        real<lower=0> a;
        real<lower=0, upper=1> beta;
        real<lower=0> tau;
    }

    model {
        // Priors
        v2~ normal(0.567,0.132); //prior for drift rate parameter v
        v1~ normal(0.529,0.116);
        a ~ normal(1.5,0.5); //prior for boundary seperation
        beta ~ normal(0.5, 0.1);
        tau ~ normal (0.3,0.1);
        // Likelihood
        for (n in 1:N) {
            // Condition 1
            if (condition[n] == 1) {
                if (choice[n] == 1) {
                    y[n] ~ normal(v1, beta); //correct
                }
                else {
                    y[n] ~ normal(v1, beta); //incorrect
                }
            }
            // Condition 2
            if (condition[n] == 2) {
                if (choice[n] == 1) {
                    y[n] ~ normal(v2, beta); //correct
                }
                else {
                    y[n] ~ normal(v2, beta); //incorrect
                }
            }
        }
    }
"""

In [11]:
def visualize_data(data, figsize=(10, 5)):
    """Helper function to visualize a simple response time data set."""

    f, axarr = plt.subplots(1, 2, figsize=figsize)
    
    # Histogram of response times
    sns.histplot(
        data[:, 0][data[:, 1] == 1], ax=axarr[0], color='#AA0000', alpha=0.8, lw=2, label=f'Response 1')
    sns.histplot(
        data[:, 0][data[:, 1] == 0], ax=axarr[0], color='#0000AA', alpha=0.8, lw=2, label=f'Response 0')

    # Barplot of categorical responses
    response, frequency = np.unique(data[:, 1], return_counts=True)
    sns.barplot(x=response.astype(np.int32), y=frequency, ax=axarr[1], alpha=0.8, color='#00AA00')

    # Labels and embelishments
    axarr[0].set_xlabel('Response time (s)', fontsize=16)
    axarr[0].legend(fontsize=16)
    axarr[0].set_ylabel('Count', fontsize=16)
    axarr[1].set_xlabel('Response', fontsize=16)
    axarr[1].set_ylabel('Frequency', fontsize=16)
    for ax in axarr:
        sns.despine(ax=ax)
        ax.grid(alpha=0.1, color='black')

    f.suptitle('Data Summary', fontsize=18)

    f.tight_layout()

In [25]:
##Calculates standard deviation
mean_a=df['condition'].mean()
std_deva=df['condition'].std()
print(mean_a)
print(std_deva)

1.5
0.5008354224706334


In [26]:
model = pystan.StanModel(model_code=program_code)
##data preparation
data={
    'N':len(df),
    'y':df['rt'],
    'condition':df['condition'],
    'choice': df['choice']
}
##Run model for analysis
Output=model.sampling(data=data,iter=1000,chains=4)
##Print to test code
print(Output)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5839e4c6a8e289dabd2386c961bc4946 NOW.


Inference for Stan model: anon_model_5839e4c6a8e289dabd2386c961bc4946.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
v1     0.53  2.0e-4   0.01   0.51   0.52   0.53   0.54   0.55   2897    1.0
v2     0.57  2.0e-4   0.01   0.55   0.56   0.57   0.57   0.59   2704    1.0
a      1.53    0.01   0.48   0.59    1.2   1.51   1.86   2.47   2141    1.0
beta   0.13  9.6e-5 5.3e-3   0.12   0.12   0.13   0.13   0.14   3025    1.0
tau     0.3  2.9e-3    0.1   0.09   0.24    0.3   0.37   0.49   1168    1.0
lp__ 460.79    0.07    1.7 456.57 459.87 461.15 462.08 463.04    660    1.0

Samples were drawn using NUTS at Sun Mar 17 17:00:42 2024.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
